# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas.api.types
import math
from scipy.stats import zscore

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier, XGBRegressor
from lifelines.utils import concordance_index
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score
from imblearn.under_sampling import RandomUnderSampler

# JCOPML Package
from jcopml.plot import plot_missing_value
from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.tuning import random_search_params as rsp
from jcopml.feature_importance import mean_loss_decrease
from jcopml.feature_importance import mean_score_decrease
from jcopml.tuning import bayes_search_params as bsp
from jcopml.tuning import grid_search_params as gsp

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
data = pd.read_csv("hasil_clustering.csv")
data

,Age_Group,Gender,Ethnicity,Smoking_Status,Alcohol_Consumption,Diet_Quality,Cholesterol_Level,Blood_Pressure,BMI,Physical_Activity,Stress_Level,Family_History,Diabetes,Air_Quality_Index,Income_Level,Sleep_Hours,Heart_Rate,Medication_Status,Cluster
0,0,0,Hispanic,1,1,0,138,100,37.7,124,1,0,0,Good,High,7.3,80,0,0
1,0,1,Hispanic,2,1,1,238,150,39.0,182,1,1,0,Good,Medium,7.8,85,0,1
2,0,1,Hispanic,2,1,2,164,106,20.7,191,2,1,1,Good,Low,5.2,61,0,0
3,1,1,White,2,1,1,158,110,37.7,36,1,1,0,Good,Medium,8.4,56,1,0
4,0,1,Black,2,1,1,282,118,30.3,237,1,0,1,Good,Low,6.7,78,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0,Native American,0,1,1,276,122,26.4,22,2,0,1,Moderate,Low,8.1,66,1,0
99996,0,0,Native American,2,1,0,225,127,23.7,203,2,0,1,Good,High,7.9,94,0,0
99997,0,1,White,2,1,0,150,130,21.5,229,0,0,0,Moderate,Low,7.4,91,1,0
99998,0,0,Asian,1,1,0,220,165,28.5,30,2,0,1,Unhealthy,Low,8.6,52,0,1


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:
data.select_dtypes(include=["object"]).columns

Index(['Ethnicity', 'Smoking_Status', 'Air_Quality_Index', 'Income_Level'], dtype='object')

In [3]:
# Label Encoding
label_encoder = LabelEncoder()

categorical_columns = data.select_dtypes(include=["object"]).columns
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column])

data.head()

,Age_Group,Gender,Ethnicity,Smoking_Status,Alcohol_Consumption,Diet_Quality,Cholesterol_Level,Blood_Pressure,BMI,Physical_Activity,Stress_Level,Family_History,Diabetes,Air_Quality_Index,Income_Level,Sleep_Hours,Heart_Rate,Medication_Status,Cluster
0,0,0,2,1,1,0,138,100,37.7,124,1,0,0,0,0,7.3,80,0,0
1,0,1,2,2,1,1,238,150,39.0,182,1,1,0,0,2,7.8,85,0,1
2,0,1,2,2,1,2,164,106,20.7,191,2,1,1,0,1,5.2,61,0,0
3,1,1,4,2,1,1,158,110,37.7,36,1,1,0,0,2,8.4,56,1,0
4,0,1,1,2,1,1,282,118,30.3,237,1,0,1,0,1,6.7,78,0,0


In [4]:
X = data.drop(columns="Cluster")
y = data["Cluster"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((80000, 18), (20000, 18), (80000,), (20000,))

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [9]:
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(scaling="minmax"), ['Age_Group', 'Gender', 'Ethnicity', 'Smoking_Status', 'Alcohol_Consumption', 'Diet_Quality', 
                                             'Cholesterol_Level', 'Blood_Pressure', 'BMI', 'Physical_Activity', 'Stress_Level', 'Family_History', 
                                             'Diabetes', 'Air_Quality_Index', 'Income_Level', 'Sleep_Hours', 'Heart_Rate', 'Medication_Status']),
    # ('categoric', cat_pipe(encoder="onehot"), ['Ethnicity', 'Smoking_Status', 'Income_Level'])
])

In [10]:
knn = Pipeline([
    ("prep", preprocessor),
    ("algo", KNeighborsClassifier(n_neighbors=5))
])

print(knn.fit(X_train, y_train))
print(knn.score(X_train, y_train))
print(knn.score(X_val, y_val))

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age_Group', 'Gender',
                                                   'Ethnicity',
                                                   'Smoking_Status',
                                                   'Alcohol_Consumption',
                                                   'Diet_Quality',
                                                   'Cholesterol_Level',
                                                   'Blood_Pressure', 'BMI',
                                                   'Physical_Activity',
   

In [11]:
rf = Pipeline([
    ("prep", preprocessor),
    ("algo", RandomForestClassifier(n_estimators=100))
])

print(rf.fit(X_train, y_train))
print(rf.score(X_train, y_train))
print(rf.score(X_val, y_val))

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age_Group', 'Gender',
                                                   'Ethnicity',
                                                   'Smoking_Status',
                                                   'Alcohol_Consumption',
                                                   'Diet_Quality',
                                                   'Cholesterol_Level',
                                                   'Blood_Pressure', 'BMI',
                                                   'Physical_Activity',
   

In [12]:
xgb = Pipeline([
    ("prep", preprocessor),
    ("algo", XGBClassifier(objective='multi:softmax', num_class=3))
])

print(xgb.fit(X_train, y_train))
print(xgb.score(X_train, y_train))
print(xgb.score(X_val, y_val))

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Age_Group', 'Gender',
                                                   'Ethnicity',
                                                   'Smoking_Status',
                                                   'Alcohol_Consumption',
                                                   'Diet_Quality',
                                                   'Cholesterol_Level',
                                                   'Blood_Pressure', 'BMI',
                                                   'Physical_Activity',
   

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [13]:
print("KNN Evaluation : ")
y_val_pred_knn = knn.predict(X_val)
report_knn = classification_report(y_val, y_val_pred_knn)
print("Classification Report:")
print(report_knn)

cm_knn = confusion_matrix(y_val, y_val_pred_knn)
print("\nConfusion Matrix KNN :")
print(cm_knn)
print("KNN Evaluation :")
print(f"Precision (macro): {precision_score(y_val, y_val_pred_knn, average='macro', zero_division=0):.4f}")
print(f"Recall (macro): {recall_score(y_val, y_val_pred_knn, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_val, y_val_pred_knn, average='macro', zero_division=0):.4f}")


print("\n")
print("Random Forest Evaluation : ")
y_val_pred_rf = rf.predict(X_val)
report_rf = classification_report(y_val, y_val_pred_rf)
print("Classification Report:")
print(report_rf)

cm_rf = confusion_matrix(y_val, y_val_pred_rf)
print("\nConfusion Matrix Random Forest :")
print(cm_rf)
print("\nRandom Forest Evaluation :")
print(f"Precision (macro): {precision_score(y_val, y_val_pred_rf, average='macro', zero_division=0):.4f}")
print(f"Recall (macro): {recall_score(y_val, y_val_pred_rf, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_val, y_val_pred_rf, average='macro', zero_division=0):.4f}")

print("\n")
print("XGB Evaluation : ")
y_val_pred_xgb = xgb.predict(X_val)
report_xgb = classification_report(y_val, y_val_pred_xgb)
print("Classification Report:")
print(report_xgb)

cm_xgb = confusion_matrix(y_val, y_val_pred_xgb)
print("\nConfusion Matrix XGB :")
print(cm_xgb)
print("\nXGB Evaluation :")
print(f"Precision (macro): {precision_score(y_val, y_val_pred_xgb, average='macro', zero_division=0):.4f}")
print(f"Recall (macro): {recall_score(y_val, y_val_pred_xgb, average='macro'):.4f}")
print(f"F1-Score (macro): {f1_score(y_val, y_val_pred_xgb, average='macro', zero_division=0):.4f}")

KNN Evaluation : 
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      8790
           1       0.89      0.90      0.89      9217
           2       1.00      1.00      1.00      1993

    accuracy                           0.90     20000
   macro avg       0.93      0.93      0.93     20000
weighted avg       0.90      0.90      0.90     20000


Confusion Matrix KNN :
[[7807  983    0]
 [ 963 8254    0]
 [   0    0 1993]]
KNN Evaluation :
Precision (macro): 0.9279
Recall (macro): 0.9279
F1-Score (macro): 0.9279


Random Forest Evaluation : 
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8790
           1       1.00      1.00      1.00      9217
           2       1.00      1.00      1.00      1993

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      

| **Model**         | **Accuracy** | **F1-Score** | **Precision** | **Recall** |
|--------------------|--------------|--------------|---------------|------------|
| **KNN**            | 0.90         | 0.9279       | 0.9279          | 0.9279      |
| **Random Forest**  | 1.00         | 1.0000        | 1.0000          | 1.0000       |
| **XGBoost**        | 1.00         | 1.0000       | 1.0000        | 1.0000        |

**Analisis Perbandingan**

**1. Accuracy**
- Random Forest dan XGBoost memiliki Accuracy sempurna (1.00), yang menunjukkan bahwa kedua model ini berhasil memprediksi semua sampel dengan benar. Ini adalah hasil yang sangat baik, tetapi perlu diwaspadai kemungkinan overfitting.
- KNN memiliki Accuracy 0.90, yang masih cukup baik tetapi lebih rendah dibandingkan Random Forest dan XGBoost. Ini menunjukkan bahwa KNN mungkin kurang cocok untuk dataset ini atau memerlukan penyesuaian parameter lebih lanjut.

**2. F1-Score**
- Random Forest dan XGBoost memiliki F1-Score sempurna (1.0000), yang menunjukkan keseimbangan sempurna antara Precision dan Recall. Ini berarti kedua model ini sangat baik dalam memprediksi semua kelas.
- KNN memiliki F1-Score 0.9279, yang menunjukkan performa yang sangat baik tetapi tidak sebaik Random Forest dan XGBoost. Ini menunjukkan bahwa KNN mungkin kurang efektif dalam menangani ketidakseimbangan kelas atau kompleksitas data.

**3. Precision**
- Random Forest dan XGBoost memiliki Precision sempurna (1.0000), yang berarti tidak ada prediksi salah untuk kelas positif.
- KNN memiliki Precision 0.9279, yang menunjukkan bahwa model ini masih memiliki beberapa kesalahan dalam memprediksi kelas positif.

**4. Recall**
- Random Forest dan XGBoost memiliki Recall sempurna (1.0000), yang berarti semua instance positif terdeteksi dengan benar.
- KNN memiliki Recall 0.9279, yang menunjukkan bahwa model ini masih melewatkan beberapa instance positif.

**Kesimpulan Perbandingan**
- andom Forest dan XGBoost memberikan hasil yang identik dan sempurna untuk semua metrik, sehingga dapat dikatakan sebagai model terbaik dalam kasus ini. Namun, hasil yang sempurna ini perlu diwaspadai karena kemungkinan overfitting.
- KNN juga memberikan performa yang sangat baik tetapi tidak sebaik Random Forest dan XGBoost. Untuk dataset atau skenario lain, model ini mungkin perlu penyesuaian parameter atau metode tambahan untuk meningkatkan performa.

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

#### Menggunakan Hyperparameter RandomizedSearchCV

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

**Berdasarkan hasil evaluasi model yang diberikan, berikut adalah analisis kelemahan model, identifikasi overfitting/underfitting, dan rekomendasi tindakan lanjutan :**

**1. Analisis Kelemahan Model**

- **KNN**
    - **Precision: 0.9279**
        - Precision sudah cukup tinggi, tetapi masih ada ruang untuk meningkatkan kemampuan model dalam meminimalkan prediksi salah pada beberapa kelas.
    - **Recall: 0.9279**
        - Recall mendekati tinggi, tetapi tidak sebaik performa dari Random Forest dan XGBoost.
    - **F1-Score: 0.9279**
        - F1-Score menunjukkan bahwa meskipun performa KNN bagus, model ini cenderung lebih sensitif terhadap perubahan pada distribusi data dibandingkan model berbasis pohon.

- **Random Forest**
    - **Precision, Recall, dan F1-Score: 1.0000**
        - Model ini tidak menunjukkan kelemahan yang berarti dalam hasil yang diperoleh. Namun, kemungkinan overfitting dapat menjadi perhatian karena akurasi sempurna di dataset validasi mungkin tidak mencerminkan performa yang sama di dataset baru.

- **XGBOOST**
    - **Precision, Recall, dan F1-Score: 1.0000**
        - Mirip dengan Random Forest, hasil yang sempurna dapat menunjukkan overfitting jika performa serupa tidak tercapai di data yang belum terlihat. Pengaturan parameter seperti regularisasi dapat membantu memastikan generalisasi yang lebih baik.

**2. Identifikasi Overfitting atau Underfitting**

- **KNN**
    - **Train Score: 0.9461**
    - **Test Score: 0.9027**
    - **Analisis:**
        - Perbedaan antara train score dan test score adalah 0.0434, yang menunjukkan bahwa model tidak mengalami overfitting yang signifikan.
        - Namun, nilai test score yang lebih rendah dibandingkan train score menunjukkan bahwa model kurang mampu menangkap pola yang lebih kompleks, mengindikasikan potensi underfitting.
        - KNN mungkin terlalu sederhana untuk dataset ini atau memerlukan penyesuaian hyperparameter (seperti n_neighbors) untuk meningkatkan performa.
    - **Kesimpulan: Model KNN cenderung underfitting.**

- **Random Forest**
    - **Train Score: 1.0**
    - **Test Score: 1.0**
    - **Analisis:**
        - Tidak ada perbedaan antara train score dan test score, yang menunjukkan model memiliki performa sempurna pada kedua dataset.
        - Hasil ini bisa disebabkan oleh dataset yang relatif sederhana atau fitur yang sangat informatif, sehingga model dapat mempelajari pola dengan sempurna tanpa overfitting.
        - Namun, perlu diwaspadai bahwa performa sempurna seperti ini jarang terjadi pada dataset dunia nyata yang lebih kompleks dan berisik.
    - **Kesimpulan: Model Random Forest tidak menunjukkan tanda-tanda overfitting atau underfitting pada dataset ini.**

- **XGBOOST**
    - **Train Score: 1.0**
    - **Test Score: 1.0**
    - **Analisis:**
        - Sama seperti Random Forest, model ini menunjukkan performa sempurna tanpa adanya perbedaan antara train dan test score.
    - **Kesimpulan: Model XGBoost tidak menunjukkan tanda-tanda overfitting atau underfitting pada dataset ini.**

**3. Rekomendasi Tindakan Lanjutan**

- **Untuk KNN (Underfitting)**
    -  **Lakukan Pencarian Hyperparameter:**
        - Cari jumlah tetangga optimal (n_neighbors) menggunakan teknik seperti Grid Search atau Random Search.
        - Pertimbangkan untuk menggunakan metrik jarak yang berbeda (misalnya, Manhattan distance atau Minkowski distance).

- **Untuk Random Forest (Tidak Overfitting/Underfitting)**
    - **Verifikasi Generalisasi:**
        - Meskipun performa pada data pelatihan dan pengujian sempurna, uji model pada dataset baru yang belum pernah dilihat selama pelatihan atau pengujian. Ini untuk memastikan bahwa model dapat generalisasi dengan baik ke data baru.
    - **Periksa Distribusi Data:**
        - Pastikan bahwa data pelatihan dan data uji memiliki distribusi yang serupa. Jika data uji terlalu mirip dengan data pelatihan, performa sempurna mungkin tidak mencerminkan kemampuan generalisasi yang sebenarnya.
    - **Pertahankan Kompleksitas Model:**
        - Karena model sudah menunjukkan performa yang sangat baik, tidak perlu melakukan perubahan besar. Namun, tetap pantau performa saat diimplementasikan pada data baru.

- **Untuk XGBoost (Tidak Overfitting/Underfitting)**
    - **Pertahankan Parameter Saat Ini:**
        - Karena model sudah menunjukkan performa sempurna, tidak perlu melakukan perubahan besar pada parameter.
    - **Evaluasi dengan Data Baru:**
        - Uji model pada dataset baru atau data yang belum terlihat untuk memastikan bahwa performa yang sempurna tidak hanya terjadi pada data pelatihan.
    - **Pantau Performa Secara Berkala:**
        - Saat diimplementasikan pada data baru, pantau performa model secara berkala untuk memastikan bahwa model tetap efektif.